In [1]:
!pip install --quiet --upgrade pip


In [5]:
# Israeli ID Card Validator using LLM
# This notebook uses OpenAI's GPT-4 Vision to analyze images and determine if they contain valid Israeli ID cards

import os
import base64
import requests
from PIL import Image
import io
from pathlib import Path

# Required installations (run in notebook cell):
# !pip install openai pillow requests

def encode_image_to_base64(image_path):
    """Convert image file to base64 string for API submission"""
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except Exception as e:
        print(f"Error encoding image: {e}")
        return None

def resize_image_if_needed(image_path, max_size=(1024, 1024)):
    """Resize image if it's too large to reduce API costs"""
    try:
        with Image.open(image_path) as img:
            if img.size[0] > max_size[0] or img.size[1] > max_size[1]:
                img.thumbnail(max_size, Image.Resampling.LANCZOS)
                # Save resized image temporarily
                temp_path = "temp_resized.jpg"
                img.save(temp_path, "JPEG", quality=85)
                return temp_path
        return image_path
    except Exception as e:
        print(f"Error resizing image: {e}")
        return image_path

def analyze_israeli_id_card(image_path, api_key):
    """
    Analyze image to determine if it contains a valid Israeli ID card
    
    Args:
        image_path (str): Path to the image file
        api_key (str): OpenAI API key
    
    Returns:
        dict: Analysis results including validity, confidence, and reasoning
    """
    
    # Resize image if needed
    processed_image_path = resize_image_if_needed(image_path)
    
    # Encode image to base64
    base64_image = encode_image_to_base64(processed_image_path)
    
    if not base64_image:
        return {"error": "Failed to process image"}
    
    # Clean up temporary file if created
    if processed_image_path != image_path and os.path.exists(processed_image_path):
        os.remove(processed_image_path)
    
    # Prepare the prompt for Israeli ID card detection
    prompt = """
    Analyze this image carefully to determine if it contains a valid Israeli ID card (תעודת זהות).
    
    Look for these specific characteristics of Israeli ID cards:
    1. Hebrew text "תעודת זהות" (Identity Card) at the top
    2. Israeli government emblem/symbol
    3. Blue and white color scheme typical of Israeli documents
    4. Personal photo on the left side
    5. Hebrew and/or Arabic text
    6. ID number (9 digits)
    7. Personal details like name, date of birth, address
    8. Security features like holograms or special printing
    9. Official formatting and layout typical of Israeli IDs
    10. Issue and expiration dates
    
    Respond with a JSON-like structure containing:
    - "is_valid_israeli_id": boolean (true/false)
    - "confidence_score": number between 0-100
    - "detected_features": list of Israeli ID features you can identify
    - "missing_features": list of expected features that are missing
    - "reasoning": detailed explanation of your analysis
    - "warnings": any concerns about image quality, partial visibility, or potential issues
    
    Be thorough and explain your reasoning clearly.
    """
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    payload = {
        "model": "gpt-4o",  # Using GPT-4 with vision capabilities
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 1000
    }
    
    try:
        response = requests.post("https://api.openai.com/v1/chat/completions", 
                               headers=headers, json=payload, timeout=30)
        response.raise_for_status()
        
        result = response.json()
        analysis = result['choices'][0]['message']['content']
        
        return {
            "success": True,
            "analysis": analysis,
            "usage": result.get('usage', {}),
            "image_processed": True
        }
        
    except requests.exceptions.RequestException as e:
        return {"error": f"API request failed: {e}"}
    except Exception as e:
        return {"error": f"Analysis failed: {e}"}

def validate_image_file(file_path):
    """Validate that the file exists and is a supported image format"""
    if not os.path.exists(file_path):
        return False, "File does not exist"
    
    try:
        with Image.open(file_path) as img:
            # Check if it's a valid image
            img.verify()
        return True, "Valid image file"
    except Exception as e:
        return False, f"Invalid image file: {e}"

def main():
    """Main function to run the Israeli ID card validator"""
    
    print("🇮🇱 Israeli ID Card Validator")
    print("=" * 40)
    
    # Get OpenAI API key
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        api_key = input("Enter your OpenAI API key: ").strip()
        if not api_key:
            print("❌ API key is required!")
            return
    
    # Get image file path
    image_path = input("Enter the path to your image file: ").strip()
    
    # Remove quotes if present
    image_path = image_path.strip('"\'')
    
    # Validate image file
    is_valid, message = validate_image_file(image_path)
    if not is_valid:
        print(f"❌ {message}")
        return
    
    print(f"✅ {message}")
    print("🔍 Analyzing image...")
    
    # Analyze the image
    result = analyze_israeli_id_card(image_path, api_key)
    
    if "error" in result:
        print(f"❌ Error: {result['error']}")
        return
    
    print("\n" + "=" * 50)
    print("📊 ANALYSIS RESULTS")
    print("=" * 50)
    print(result['analysis'])
    
    if 'usage' in result and result['usage']:
        usage = result['usage']
        print(f"\n💰 API Usage:")
        print(f"   Prompt tokens: {usage.get('prompt_tokens', 'N/A')}")
        print(f"   Completion tokens: {usage.get('completion_tokens', 'N/A')}")
        print(f"   Total tokens: {usage.get('total_tokens', 'N/A')}")

# Example usage in Jupyter notebook:
if __name__ == "__main__":
    main()

# Alternative function for direct use in notebook cells:
def quick_validate(image_path, api_key=None):
    """Quick validation function for notebook use"""
    if not api_key:
        api_key = os.getenv('OPENAI_API_KEY')
        if not api_key:
            print("Please set OPENAI_API_KEY environment variable or pass api_key parameter")
            return None
    
    result = analyze_israeli_id_card(image_path, api_key)
    if "error" in result:
        print(f"Error: {result['error']}")
        return None
    
    print("Analysis Results:")
    print("-" * 30)
    print(result['analysis'])
    return result

# Example usage in notebook:
# result = quick_validate("path/to/your/image.jpg")

# For batch processing multiple images:
def batch_validate(image_folder, api_key=None):
    """Validate multiple images in a folder"""
    if not api_key:
        api_key = os.getenv('OPENAI_API_KEY')
        if not api_key:
            print("Please set OPENAI_API_KEY environment variable")
            return
    
    folder_path = Path(image_folder)
    if not folder_path.exists():
        print(f"Folder {image_folder} does not exist")
        return
    
    # Supported image extensions
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff'}
    
    image_files = [f for f in folder_path.iterdir() 
                   if f.suffix.lower() in image_extensions]
    
    if not image_files:
        print(f"No image files found in {image_folder}")
        return
    
    results = {}
    print(f"Processing {len(image_files)} images...")
    
    for i, image_file in enumerate(image_files, 1):
        print(f"\n[{i}/{len(image_files)}] Processing: {image_file.name}")
        
        result = analyze_israeli_id_card(str(image_file), api_key)
        results[image_file.name] = result
        
        if "error" not in result:
            print("✅ Analysis completed")
        else:
            print(f"❌ Error: {result['error']}")
    
    return results

🇮🇱 Israeli ID Card Validator
✅ Valid image file
🔍 Analyzing image...

📊 ANALYSIS RESULTS
```json
{
  "is_valid_israeli_id": false,
  "confidence_score": 95,
  "detected_features": [],
  "missing_features": [
    "Hebrew text 'תעודת זהות' at the top",
    "Israeli government emblem/symbol",
    "Blue and white color scheme typical of Israeli documents",
    "Personal photo on the left side",
    "Hebrew and/or Arabic text",
    "ID number (9 digits)",
    "Personal details like name, date of birth, address",
    "Security features like holograms or special printing",
    "Official formatting and layout typical of Israeli IDs",
    "Issue and expiration dates"
  ],
  "reasoning": "The image shows a child and several products in a store environment. There is no visible ID card or document resembling an Israeli ID card present in the image. The features typical of an Israeli ID, such as Hebrew text, government emblem, and personal details, are not present.",
  "warnings": "The image does n